# Calculating a Zero Beta Portfolio #

### First Approach: Five Stocks ###

In [2]:
# Import Libraries

# Data Management
import pandas as pd
import numpy as np

# Plots
import matplotlib.pyplot as plt

# Statistics
import statsmodels.api as sm

# Pretty Notation
from IPython.display import display, Math

In [3]:
def import_financial_data(
    ticker: str
):

    # Check the ticker for Upper Cases
    ticker = ticker if ticker.isupper() else ticker.upper()

    # Import data
    df = pd.read_csv(rf"..\stocks\{ticker}.csv")

    # Set the Index
    df = df.set_index('Date')
    df.index = pd.to_datetime(df.index)

    df_useful_data = df[['Open Price', 'High Price', 'Low Price', 'Close Price', 'Adjusted_close']]

    df_useful_data = df_useful_data.rename(columns={
        "Open Price":"open",
        "High Price":"high",
        "Low Price":"low",
        "Close Price":"close",
        "Adjusted_close":"adjusted_close",
    })

    # Drop NaN's
    df_useful_data.dropna(inplace = True)

    return df_useful_data.loc["2015-01-01":]

In [4]:
# Get the important data for the Risk Free Rate

rfr = pd.read_csv(rf"..\additional_data\rfr.csv")
rfr = rfr.set_index('Date')
rfr.index = pd.to_datetime(rfr.index, dayfirst=True)
rfr.dropna(inplace = True)

rfr

In [5]:
# Get the important data for the S&P500

sp500 = pd.read_csv(rf"..\additional_data\sp500.csv")
sp500 = sp500.set_index('Date')
sp500.index = pd.to_datetime(sp500.index)

sp500

In [6]:
# Get the important data for the S&P500

betas_df = pd.read_csv(rf"..\additional_data\betas.csv")
betas_df = betas_df.set_index('Date')
betas_df.index = pd.to_datetime(betas_df.index)

betas_df

In [7]:
# Import Data

# Apple Data
df_1 = import_financial_data('AAPL')

# Amazon Data
df_2 = import_financial_data('AMZN')

# Meta Data
df_3 = import_financial_data('META')

# Microsoft Data
df_4 = import_financial_data('MSFT')

# Nvidia Data
df_5 = import_financial_data('NVDA')


In [8]:
# Let us calculate the Betas for some stocks

# Apple
apple_beta = betas_df['AAPL']

# Amazon
amazon_beta = betas_df['AMZN']

# META
meta_beta = betas_df['META']

# Microsoft
microsoft_beta = betas_df['MSFT']

# NVDIA
nvdia_beta = betas_df['NVDA']

In [9]:
# Create Plot

plt.figure(figsize=(10, 6))
plt.plot(apple_beta, label='Apple Beta', color='blue', alpha=0.7)
plt.plot(amazon_beta, label='Amazon Beta', color='red', alpha=0.7)
plt.plot(meta_beta, label='META Beta', color='green', alpha=0.7)
plt.plot(microsoft_beta, label='Microsoft Beta', color='orange', alpha=0.7)
plt.plot(nvdia_beta, label='NVDIA Beta', color='purple', alpha=0.7)
plt.axhline(y=1, color='black', linestyle='dashed')

# Config
plt.title('Beta Time Series')
plt.xlabel('Time')
plt.ylabel('Beta')
plt.legend()

# Show
plt.show()

In [10]:
# Let us find an unsensible portfolio to the market

data_portfolio = pd.DataFrame()

data_portfolio['AAPL'] = df_1['adjusted_close']
data_portfolio['AMZN'] = df_2['adjusted_close']
data_portfolio['META'] = df_3['adjusted_close']
data_portfolio['MSFT'] = df_4['adjusted_close']
data_portfolio['NVDA'] = df_5['adjusted_close']

data_portfolio = data_portfolio.dropna()

data_portfolio = data_portfolio.loc['2016-01-01':]

data_portfolio

In [11]:
# Returns

df_returns = data_portfolio.pct_change(1).dropna().mul(100)

df_returns

In [12]:
# Common Index

common_index = df_returns.index.intersection(betas_df.index)  # Common DATE
df_returns = df_returns.reindex(common_index)
betas_df = betas_df.reindex(common_index)

In [13]:
# We have to find the mean expected returns

expected_returns = df_returns.mean()

expected_returns

In [14]:
# We have to also find the standard deviations

volat = df_returns.dropna().std()

volat

In [15]:
# Also we have to find the covariance matrix

cov_matrix = df_returns.dropna().cov()

cov_matrix

In [16]:
# We need also to find the mean betas of the stocks

sample_beta_df = pd.DataFrame()

sample_beta_df['AAPL'] = apple_beta
sample_beta_df['AMZN'] = amazon_beta
sample_beta_df['META'] = meta_beta
sample_beta_df['MSFT'] = microsoft_beta
sample_beta_df['NVDA'] = nvdia_beta

mean_betas = sample_beta_df.mean()

mean_betas

In [17]:
# Which is the form of the portfolio weights if we want a zero-beta portfolio

display(Math(r"\omega=\left(\frac{D}{DC-E^2}\right)\Sigma^{-1}\iota+\left(\frac{E}{DC-E^2}\right)\Sigma^{-1}\beta"))
display(Math(r"C = \iota^⊤\Sigma^{-1}\iota"))
display(Math(r"D = \beta^⊤\Sigma^{-1}\beta"))
display(Math(r"E = \beta^⊤\Sigma^{-1}\iota"))
display(Math(r"\Delta = DC-E^2"))

In [18]:
# So let us obtain the components

beta = mean_betas.values.flatten().reshape(-1, 1)
Sigma = cov_matrix.values

lambda_ = 1e-6  # Tikhonov Regularization
Sigma_inv = np.linalg.inv(Sigma + lambda_ * np.eye(Sigma.shape[0]))

iota = np.ones((5, 1))

In [19]:
# And now obtain the coefficients of the Efficient Frontier

C = np.dot(np.dot(iota.T, Sigma_inv), iota)
D = np.dot(np.dot(beta.T, Sigma_inv), beta)
E = np.dot(np.dot(beta.T, Sigma_inv), iota)
Delta = (D*C - E*E)

print(f"This is C: {C}")
print(f"This is D: {D}")
print(f"This is E: {E}")
print(f"This is Delta: {Delta}")

In [20]:
# Now the weights

beta_weights = ((D/Delta)*(Sigma_inv @ iota)) - ((E/Delta)*(Sigma_inv @ beta))

beta_weights

In [21]:
# Create the portfolio

df_returns["zero_beta"] = df_returns.iloc[:, :5] @ beta_weights

df_returns

In [22]:
df_returns["risk_free_rate"] = (((1 + (rfr.div(100)))**(1/360)) - 1).mul(100)
df_returns['market_returns'] = sp500.pct_change(1).mul(100)

df_returns

In [23]:
# Create the excess returns

df_returns['portfolio_excess'] = df_returns['zero_beta'] - df_returns['risk_free_rate']
df_returns['market_excess'] = df_returns['market_returns'] - df_returns['risk_free_rate']

df_returns

In [24]:
# Calculate the beta with the weights

zero_beta_alt = mean_betas.transpose().dot(beta_weights)

print(zero_beta_alt.round(4))

In [25]:
def calculate_analytics(df_returns, risk_free_rate=0.0):
    # Trading Days in one Year
    ann_factor = 252  
    
    # Annualized Returns
    annualized_return = df_returns.mean() * ann_factor
    
    # Annualized Volatility
    annualized_std = df_returns.std() * np.sqrt(ann_factor)
    
    # Sharpe Ratio
    sharpe_ratio = (annualized_return - risk_free_rate) / annualized_std
    
    # Max Drawdown
    cumulative_returns = (1 + df_returns.div(100)).cumprod()
    rolling_max = cumulative_returns.cummax()
    drawdown = (cumulative_returns / rolling_max) - 1
    max_drawdown = drawdown.min()

    # VaR at 95%
    var_95 = df_returns.quantile(0.05)

    # Create DF
    summary_df = pd.DataFrame({
        "Annualized Returns": annualized_return,
        "Annualized Volatility": annualized_std,
        "Sharpe Ratio": sharpe_ratio,
        "Max Drawdown": max_drawdown,
        "VaR 95%": var_95
    })
    
    return summary_df

In [26]:
# Create a DataFrame

tickers = ["AAPL", "AMZN", "META", "MSFT", "NVDA", "zero_beta"]

df_portfolios = df_returns[tickers]

df_portfolios

In [27]:
# And the Table

analytics_table = calculate_analytics(df_portfolios)

analytics_table

In [28]:
# Time Series Graphs
plt.figure(figsize=(10, 6))
plt.plot(df_returns['AAPL'].cumsum(), label='APPL Price', color='blue', alpha=0.7)
plt.plot(df_returns['AMZN'].cumsum(), label='AMZN Price', color='green', alpha=0.7)
plt.plot(df_returns['META'].cumsum(), label='META Price', color='red', alpha=0.7)
plt.plot(df_returns['MSFT'].cumsum(), label='MSFT Price', color='orange', alpha=0.7)
plt.plot(df_returns['NVDA'].cumsum(), label='NVDA Portfolio', color='purple', alpha=0.7)
plt.plot(df_returns['zero_beta'].cumsum(), label='Zero Beta Portfolio', color='black', alpha=1)

# Config
plt.title('Cumulative Returns Time Series')
plt.xlabel('Time Index')
plt.ylabel('Cumulative Returns')
plt.legend()

# Show
plt.grid(True)
plt.show()

In [29]:
# Create Figure
fig, ax1 = plt.subplots(dpi = 300)

# SP500 Returns
sp500['sp_500'].pct_change(1).mul(100).cumsum().plot(color = 'blue', ax = ax1)
ax1.set_xlabel('Date')
ax1.set_ylabel(
    'SP500 Returns', 
    color='blue'
    )

# Zero Beta Portfolio Returns
ax2 = ax1.twinx()

df_returns['zero_beta'].cumsum().plot(color = 'red', ax = ax2)
ax2.set_ylabel(
    'Zero Beta Portfolio Returns', 
    color='red'
    )

plt.show()

In [30]:
# Create the Weights function
def wexp(N, half_life):
    c = np.log(0.5)/half_life
    n = np.array(range(N))
    w = np.exp(c*n)
    return np.flip(w/np.sum(w))

In [31]:
# Calculate the beta

window = len(df_returns)
weights = window * wexp(window, window/2)

#Model specification
model = sm.WLS(
    df_returns['portfolio_excess'], 
    sm.add_constant(df_returns['market_excess']),
    weights = weights,
    missing='drop'
    )   
     
#the results of the model
results = model.fit() 
    
#here we check the summary
print(results.summary())       